# **Data Pre-Processing**

## **Import libraries**

In [1]:
import pandas as pd
import numpy as np
import re
import glob
import os
import pandas as pd
import re
import pandas as pd
import re
import unicodedata
from fuzzywuzzy import fuzz


## **Load data**

In [2]:
input_path = '../data/raw' 
files = glob.glob(os.path.join(input_path, "Page*to*.csv")) 

# 2. Đọc và gộp các file lại
List = []
for filename in files:
    df = pd.read_csv(filename, index_col=None, header=0)
    List.append(df)

raw = pd.concat(List, axis=0, ignore_index=True)

# 3. Xem sơ qua dữ liệu
print(f"Tổng số dòng dữ liệu: {raw.shape[0]}")
raw.head()

# 4. Lưu file gộp ra file CSV mới
output_path = '../data/raw/full.csv'

# Lưu ý quan trọng: index=False để không lưu thêm cột số thứ tự thừa
# encoding='utf-8-sig' để mở bằng Excel không bị lỗi phông tiếng Việt
raw.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"Đã lưu thành công file gộp tại: {output_path}")


Tổng số dòng dữ liệu: 24121
Đã lưu thành công file gộp tại: ../data/raw/full.csv


## **Convert price & area**

In [3]:
def covert_price(text):
    if pd.isna(text):
        return np.nan
    
    text = str(text).lower().strip()
    
    if any(word in text for word in ['thương lượng', 'liên hệ', 'call', 'zalo', 'inbox', 'lh', 'gọi']):
        return np.nan
    
    text = re.sub(r'\b(tháng|đồng|đ|/tháng|/thang|/th)\b', ' ', text)
    
    text = text.replace(',', '.')
    
    numbers = re.findall(r'\d+\.?\d*', text)
    
    if any(word in text for word in ['trieu', 'triệu', 'tr', 'triệu', 'millions?', 'm$']):
        matches = re.findall(r'(\d+\.?\d*)\s*(triệu|tr|trieu)', text)
        if matches:
            return float(matches[0][0])
        matches = re.findall(r'(\d+\.?\d*)\s*(tr|triệu)', text)
        if matches:
            return float(matches[0][0])
    
    if numbers:
        candidates = []
        for num in numbers:
            num = num.replace('.', '')
            if len(num) >= 6:  # >= 100000
                price = float(num) / 1_000_000
                candidates.append(round(price, 2))
        
        if candidates:
            return max(candidates)  # min
    
    matches = re.search(r'(\d+\.?\d*)\s*k\b', text)
    if matches:
        return round(float(matches.group(1)) / 1000, 2)
    
    matches = re.search(r'\b(\d+\.\d{1,2})\b', text)
    if matches:
        try:
            val = float(matches.group(1))
            if 0.5 <= val <= 50:  
                return val
        except:
            pass
    
    return np.nan

def covert_area(text):
    if pd.isna(text):
        return np.nan
    
    text = str(text).lower()
    
    text = re.sub(r'm²|m2|\bmét vuông\b|\bm2\b|\bm²\b', 'm2', text)
    
    matches = re.findall(r'(\d+\.?\d*)\s*m2', text)
    if matches:
        return float(matches[-1])  
    
    matches = re.findall(r'(\d+)\s*m2', text)
    if not matches:
        matches = re.findall(r'(\d+)m2', text)
    
    if matches:
        return float(matches[-1])
    
    matches = re.search(r'(?:khoảng|etwa|xấp xỉ)\s*(\d+)\s*m2', text)
    if matches:
        return float(matches.group(1))
    
    return np.nan

raw['price'] = raw['price'].apply(covert_price)
raw['area'] = raw['area'].apply(covert_area)

# raw.head(n=5)


## **Extract adddress & street name**

In [4]:
raw.rename(columns={'address': 'location'}, inplace=True)


In [5]:
def fix_unicode_vn(text):
    if pd.isna(text):
        return ""
    s = str(text)
    fixes = {
        'Ờng': 'ường', 'ờng': 'ường', 'Ường': 'ường', 'Trưường': 'Trường', 
        'ĐưỜng': 'Đường', 'đưỜng': 'đường', 'ĐƯỜNG': 'Đường',
        'Phố': 'Phố', 'phố': 'phố', 'TL': 'Tỉnh Lộ', 'QL': 'Quốc Lộ',
        'q.': 'Quận ', 'h.': 'Huyện ', 'p.': 'Phường ', 'x.': 'Xã '
    }
    for wrong, correct in fixes.items():
        s = s.replace(wrong, correct)
    return s

def clean_street(text):
    if pd.isna(text) or text == "UNKNOWN":
        return "UNKNOWN"
    
    text = str(text).strip()
    
    # Regex xóa prefix: ^(đường|phố...) ở ĐẦU CÂU, không phân biệt hoa thường
    # Thêm cả các lỗi chính tả phổ biến vào đây
    pattern = r'^(đường|phố|đưường|duong|đ|d|đg|ngõ|hẻm|ngách|tỉnh lộ|quốc lộ|tl|ql)\s+'
    
    clean_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Xóa luôn các ký tự rác nếu còn sót (ví dụ: "Thới Tam Thôn," -> "Thới Tam Thôn")
    clean_text = clean_text.strip(',. ')
    
    return clean_text.title()

# ====================== 3. HÀM EXTRACT STREET (Đã sửa lỗi Logic) ======================
def extract_street(address):
    if pd.isna(address):
        return pd.Series(["UNKNOWN", "UNKNOWN"])
    
    # Chuẩn hóa sơ bộ
    raw = str(address)
    addr = " " + fix_unicode_vn(raw).lower() + " " 
    
    house_number = "UNKNOWN"
    street_name = "UNKNOWN"
    
    # === BƯỚC 1: Tách số nhà (Logic cũ của bạn khá ổn) ===
    # Thêm case bắt "Ấp đông 5" -> coi 5 là số nhà, hoặc coi cả cụm là địa chỉ
    # Ở đây mình ưu tiên tìm số trước
    house_match = re.search(r'\b(\d+[\d\/\-\.A-Za-z]*)\b', addr)
    if house_match:
        cand = house_match.group(1)
        # Chỉ lấy nếu độ dài hợp lý (tránh số điện thoại)
        if len(cand.replace('/', '').replace('-', '').replace('.', '')) <= 10:
            house_number = cand.upper()
            # Xóa số nhà khỏi chuỗi để tránh nhiễu tên đường
            addr = addr.replace(cand, " ")

    # === BƯỚC 2: Danh sách Prefix (ĐÃ SỬA LỖI THIẾU DẤU PHẨY) ===
    street_prefixes = [
        'đường', 'duong', 'đuờng', 'đường', 'đg', 'đ',
        'phố', 'pho',
        'đại lộ', 'dai lo',
        'quốc lộ', 'ql', 'quoc lo',
        'tỉnh lộ', 'tl', 'tinh lo',
        'hẻm', 'hem',
        'ngõ', 'ngo',
        'đường số', 'duong so',
        'ấp', 'ap', # Thêm Ấp vào đây vì Hóc Môn hay dùng Ấp làm tên đường
        'thôn', 'xóm'
    ]
    
    # === BƯỚC 3: Regex tìm tên đường (ĐÃ SỬA LOOKAHEAD thêm 'xã') ===
    # (?=...) là điều kiện dừng. Mình thêm 'xã', 'thôn', 'ấp' vào để nó cắt đúng chỗ.
    stop_words = r',|\s+phường|\s+quận|\s+huyện|\s+thành phố|\s+tỉnh|\s+xã|\s+thị trấn|\s+thôn|\s+ấp|$'
    
    found = False
    for prefix in street_prefixes:
        # Regex: Tìm [Prefix] + [Tên đường] + [Dấu hiệu dừng]
        pattern = rf'{prefix}\s+([^,;\(\)\[\]]+?)(?={stop_words})'
        match = re.search(pattern, addr)
        if match:
            name = match.group(1).strip()
            # Nếu tên tìm được quá ngắn (ví dụ "số") hoặc là số, bỏ qua
            if len(name) > 1 and not name.isdigit():
                street_name = name
                found = True
                break 
    
    # === BƯỚC 4: Fallback (Nếu không tìm thấy bằng prefix) ===
    if not found:
        # Nếu có chữ "Phố" dính liền tên riêng (VD: Phố Huế)
        pho_match = re.search(r'phố\s+([a-zA-Z\sàáảãạâấầẩẫậăắằẳẵặèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđ]+)', addr)
        if pho_match:
            street_name = pho_match.group(1)
        else:
            # Lấy phần đầu tiên của địa chỉ (trước dấu phẩy đầu tiên)
            first_part = addr.split(',')[0].strip()
            # Clean các từ hành chính nếu lỡ dính vào
            for admin in ['phường', 'quận', 'huyện', 'tỉnh', 'xã']:
                if admin in first_part:
                    first_part = first_part.split(admin)[0]
            
            if len(first_part) > 2:
                street_name = first_part

    # === BƯỚC 5: FINAL CLEAN (Quan trọng nhất với bạn) ===
    # Chạy hàm clean để xóa "Đường", "Đưường" còn sót lại
    street_name = clean_street(street_name)

    return pd.Series([house_number, street_name])
def extract_address(location):
    if pd.isna(location):
        return "UNKNOWN"
    
    parts = str(location).split(',')
    if len(parts) >= 2:
        return parts[-2].strip() + ' - ' + parts[-1].strip()
    else:
        return "UNKNOWN"
    
raw[['house_number', 'street_name']] = raw['location'].apply(extract_street) 
raw['address'] = raw['location'].apply(extract_address)
# ====================== CHẠY TEST ======================
# print("Đang xử lý...")
# print("\n=== KẾT QUẢ ===")
# display(raw[['address', 'house_number', 'street_name']])


## **Adjust interior and amenities**

In [ ]:
# ====================== BƯỚC 1: Chuẩn hóa văn bản cực mạnh ======================
def normalize_text(text):
    if pd.isna(text):
        return ""
    # Chuyển về lowercase
    text = str(text).lower()
    # Chuẩn hóa unicode (đảm bảo cả có dấu và không dấu đều match được)
    text = unicodedata.normalize('NFC', text)
    # Thay thế biến thể phổ biến
    replacements = {
        'may lanh': 'máy lạnh', 'maylanh': 'máy lạnh', 'đh': 'điều hòa', 'dh': 'điều hòa',
        'ac': 'máy lạnh', 'air': 'máy lạnh', 'máy điều hoà': 'điều hòa',
        'tu lanh': 'tủ lạnh', 'tulanh': 'tủ lạnh',
        'may giat': 'máy giặt', 'maygiat': 'máy giặt',
        'gac lung': 'gác lửng', 'gac cao': 'gác lửng', 'gac xep': 'gác lửng',
        'bancol': 'ban công', 'balcony': 'ban công',
        'nha xe': 'bãi xe', 'ham xe': 'hầm xe', 'giữ xe': 'bãi xe',
        'tu ao': 'tủ quần áo', 'tu quan ao': 'tủ quần áo',
        'giuong': 'giường', 'nem': 'nệm', 'ga goi': 'ga gối'
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

# ====================== BƯỚC 2: Từ điển từ khóa SIÊU DÀI + negativeATION ======================
# Mỗi tiện ích có 3 danh sách:
# - positive: từ khóa khẳng định
# - negative: từ khóa phủ định (ví dụ: "không có máy lạnh")
# - fuzzy_threshold: ngưỡng fuzzy nếu dùng fuzzy matching
amenity_keywords = {
    'ac': {
        'positive': ['máy lạnh', 'điều hòa', 'có máy lạnh', 'có điều hòa', 'máy lạnh inverter', 'máy lạnh mới', 'điều hoà'],
        'negative': ['không có máy lạnh', 'không điều hòa', 'không máy lạnh', 'chưa có máy lạnh'],
        'fuzzy': 90
    },
    'fridge': {
        'positive': ['tủ lạnh', 'có tủ lạnh', 'tủ lạnh lớn', 'tủ lạnh riêng'],
        'negative': ['không tủ lạnh', 'không có tủ lạnh'],
        'fuzzy': 90
    },
    'washing_machine': {
        'positive': ['máy giặt', 'có máy giặt', 'máy giặt chung', 'máy giặt riêng', 'giặt sấy'],
        'negative': ['không máy giặt', 'không có máy giặt'],
        'fuzzy': 85
    },
    'mezzanine': {
        'positive': ['gác', 'gác lửng', 'có gác', 'gác cao', 'gác đúc', 'gác xép', 'duplex', 'phòng gác'],
        'negative': ['không gác', 'không có gác'],
        'fuzzy': 90
    },
    'kitchen': {
        'positive': ['bếp', 'kệ bếp', 'tủ bếp', 'nấu ăn', 'được nấu ăn', 'nấu nướng', 'bếp riêng', 'bếp từ', 'bếp điện'],
        'negative': ['không nấu ăn', 'không được nấu'],
        'fuzzy': 85
    },
    'wardrobe': {
        'positive': ['tủ quần áo', 'tủ đồ', 'tủ áo', 'có tủ quần áo'],
        'negative': [],
        'fuzzy': 90
    },
    'bed': {
        'positive': ['giường', 'nệm', 'có giường', 'giường nệm', 'giường mới'],
        'negative': [],
        'fuzzy': 90
    },
    'balcony': {
        'positive': ['ban công', 'bancol', 'balcony', 'có ban công', 'view ban công'],
        'negative': [],
        'fuzzy': 90
    },
    'elevator': {
        'positive': ['thang máy', 'có thang máy', 'tòa nhà thang máy', 'thang máy lên phòng'],
        'negative': [],
        'fuzzy': 90
    },
    'free_time': {  # Giờ giấc tự do
        'positive': ['giờ giấc tự do', 'tự do giờ giấc', 'giờ thoải mái', 'ra vào tự do', 'không chung chủ'],
        'negative': ['giờ giới nghiêm', 'giờ giấc nghiêm ngặt'],
        'fuzzy': 85
    },
    'parking': {
        'positive': ['bãi xe', 'hầm xe', 'chỗ để xe', 'giữ xe miễn phí', 'để xe trong nhà', 'hầm để xe'],
        'negative': ['không chỗ để xe', 'không để xe'],
        'fuzzy': 85
    }
}

# ====================== BƯỚC 3: Hàm kiểm tra cực mạnh ======================
def amenity(text, config):
    if pd.isna(text):
        return 0
    text = normalize_text(text)

    # 1. Kiểm tra phủ định trước → nếu có là trả 0 luôn
    for negative in config['negative']:
        if negative in text:
            return 0

    # 2. Kiểm tra từ khóa chính xác
    for positive in config['positive']:
        if positive in text:
            return 1

    # 3. Fuzzy matching (bắt các kiểu viết sai chính tả)
    for positive in config['positive']:
        clean = positive.lower()
        for word in text.split():
            if fuzz.ratio(clean, word) >= config.get('fuzzy', 85):
                return 1
            if fuzz.partial_ratio(clean, word) >= 95:
                return 1
    return 0

for col_name, config in amenity_keywords.items():
    raw[col_name] = raw['thongtinmota'].apply(lambda x: amenity(x, config))


## **Process date**

In [ ]:
def process_date(date):
    try:
        match = re.search(r'\d{2}/\d{2}/\d{4}', str(date))
        if match:
            return pd.to_datetime(match.group(), format='%d/%m/%Y')
        return pd.to_datetime(date)
    except:
        return pd.to_datetime(None)

raw['date'] = raw['ngaydang'].apply(process_date)


In [ ]:
raw.columns


## **Save result**

In [ ]:
columns = [
    'title', 'thongtinmota', 'location', 'address', 'street_name', 'price', 'area', 
    'date', 'ac', 'fridge',
    'washing_machine', 'mezzanine', 'kitchen', 'wardrobe',
    'bed', 'balcony', 'elevator', 'free_time', 'parking',
    'url'
]

cleaned = raw[columns].copy()

cleaned.rename(columns={'ac': 'air_conditioning'}, inplace=True)
cleaned.rename(columns={'thongtinmota': 'description'}, inplace=True)

output_path = '../data/cleaned/cleaned_data6.csv'
cleaned.to_csv(output_path, index=False)

# print(f"Đã lưu dữ liệu sạch vào: {output_path}")
# print(cleaned.info())
